In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
from model import Model
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging
import torch
import torch.nn as nn
import gc
from einops import rearrange
from tqdm import tqdm
# from kw_r2plus1d_video_swin_transformer import SwinTransformer3D
batchsz = 4
frame_nums=15
start_epoch = 0
test_flage=False
model_name ="face_key_point"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
# device = torch.device('cuda:1' if USE_CUDA else 'cpu')

[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.


Using cache found in /home/ssrlab/.cache/torch/hub/SharanSMenon_swin-transformer-hub_main
/home/ssrlab/ENTER/envs/qx/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
logger.debug('============={}+2D Video Vision Transformer Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2024-10-15 14:19:46,689]::2730006395::DEBUG::=============face_key_point+2D Video Vision Transformer Strat!=============


True
학습을 진행하는 기기: cuda:0


In [3]:
import numpy as np
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

num_face_keypoints = 478
num_pose_keypoints = 23
total_keypoints = num_face_keypoints + num_pose_keypoints

A = np.zeros((total_keypoints, total_keypoints))

face_connections = []
face_connections.extend(mp_face_mesh.FACEMESH_TESSELATION)  
face_connections.extend(mp_face_mesh.FACEMESH_CONTOURS)   
face_connections.extend(mp_face_mesh.FACEMESH_IRISES)      

pose_connections = [
    (11, 12),  
    (11, 13),  
    (13, 15),  
    (12, 14),  
    (14, 16),  
    (23, 24),  
    (23, 25),  
    (25, 27), 
    (24, 26),  
    (26, 28), 
    (11, 23),  
    (12, 24)   
]

for i, j in face_connections:
    A[i, j] = 1
    A[j, i] = 1  

for i, j in pose_connections:
    A[i-10 + 478, j-10 + 478] = 1
    A[j-10 + 478, i-10 + 478] = 1
A = torch.from_numpy(np.asarray(A)).to(device)

In [4]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [ ]:

save_model_file_path = './save_swintransformer_folder/{}_{}.{}'
mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, thresholds=[0.6, 0.7, 0.7], post_process=True)
# test_set_data=getdata("/home/ssrlab/qx/code/test/video-swin-transformer-pytorch/data/15Frames/test_set.dat","test")
# test_set_data=test_set_data[:datasize("test",batchsz)]
print(1)

1


In [6]:
validation_set_data=getdata("/home/ssrlab/qx/video-swin-transformer-pytorch/data/15Frames/15Frames_bodykeys_correct/valid_set.dat","valid")
# validation_set_data=reshape_to_expected_validinput(validation_set_data)
print(1)

1


In [7]:
train_set_data=getdata("/home/ssrlab/qx/video-swin-transformer-pytorch/data/15Frames/15Frames_bodykeys_correct/train_set.dat","train")
# train_set_data=reshape_to_expected_traininput(train_set_data)
print(1)

1


In [8]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [9]:
class ChalearnDataset(Dataset):
    def __init__(self,fullshot,keypoint,tagdata,transform=None):
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.keypoint=keypoint
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        # audiodata = self.audiodata[idx]
        # audiodata = torch.FloatTensor(audiodata)
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        keypoint = self.keypoint[idx]
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return fullshot,keypoint,big_five_sorces
print(1)

1


In [10]:
num_workerssz = 20
lr = 3e-05
epochs = 120
print(1)

1


In [11]:
train_set_data = ChalearnDataset(fullshot=train_set_data[1],keypoint = train_set_data[3],tagdata=train_set_data[5],transform=transform)
val_set_data = ChalearnDataset(fullshot=validation_set_data[1],keypoint = validation_set_data[3],tagdata=validation_set_data[5],transform=transform)
train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [12]:
model=Model(A, 3, 256, 256, [3, 3*3], [3*3, 16, 32, 64], 3)#adj, num_v, num_classes, gc_dims, sc_dims, feat_dims
model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

1


In [13]:
from torchinfo import summary

summary(model, input_size = ((4, 3, 224, 224),(4,15,501,3)), col_names = ['input_size','output_size','num_params'])

/home/ssrlab/qx/RGB_Skeleton/Swin-Transformer/facekeypointmodelswintransformer/model.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/torch/csrc/utils/tensor_new.cpp:261.)
  mask_code = torch.Tensor(mask_code)


Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #
Model                                                   [4, 3, 224, 224]          [4, 5]                    21,331,496
├─GGCN: 1-1                                             [4, 15, 501, 3]           [4, 256]                  15
│    └─GraphConvolution: 2-1                            [1503, 1503]              [4, 13, 1503, 9]          --
│    │    └─Dropout: 3-1                                [4, 13, 1503, 6]          [4, 13, 1503, 6]          --
│    └─StandConvolution: 2-2                            [4, 13, 1503, 9]          [4, 256]                  --
│    │    └─Dropout: 3-2                                [4, 9, 13, 1503]          [4, 9, 13, 1503]          --
│    │    └─Sequential: 3-3                             [4, 9, 13, 1503]          [4, 64, 2, 185]           13,632
│    │    └─self_attention: 3-4                         [4, 64, 2, 185]           --           

In [14]:
def extract_ocean(data,batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    if batch_size >1:
        for i in range(batch_size):
            o.append(data[i][0].item())
            c.append(data[i][1].item())
            e.append(data[i][2].item())
            a.append(data[i][3].item())
            n.append(data[i][4].item())
        o = torch.tensor(o) 
        c = torch.tensor(c) 
        e = torch.tensor(e) 
        a = torch.tensor(a) 
        n = torch.tensor(n) 
    elif batch_size == 1:
        data=data.squeeze()
        for i in range(batch_size):
            o.append(data[0].item())
            c.append(data[1].item())
            e.append(data[2].item())
            a.append(data[3].item())
            n.append(data[4].item())
            
        o = torch.tensor(o) 
        c = torch.tensor(c) 
        e = torch.tensor(e) 
        a = torch.tensor(a) 
        n = torch.tensor(n)
    return o,c,e,a,n

In [15]:
def imagecrop(imagedata,batchsz):
    # convert_tensor= transforms.ToTensor()
    output=[]
    imagedata=imagedata.permute(0,2,1,3,4)
    for j in range(batchsz):#비디오 데이터 batchsz(9) 개 있다
        firstoutput=[]
        for i in range(len(imagedata[j])):#imagedata에 15개의 프레임이 있다
            # temp = convert_tensor(imagedata[i])
            #이미지를 나누
            temp = imagedata[j][i].unfold(1,2,2).unfold(2,2,2)
            temp = rearrange(temp, 'c h1 w1 h w -> (h1 w1) c h w')
            firstoutput.append(temp)
        firstoutput = torch.stack(firstoutput,0)
        output.append(firstoutput)
    output = torch.stack(output,0)
    return output

In [16]:
torch.autograd.set_detect_anomaly(True)
with torch.cuda.device(0):
    max_value = float('inf')
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        train_avg_o_loss = 0
        train_avg_c_loss = 0
        train_avg_e_loss = 0
        train_avg_a_loss = 0
        train_avg_n_loss = 0
        val_avg_o_loss = 0
        val_avg_c_loss = 0
        val_avg_e_loss = 0
        val_avg_a_loss = 0
        val_avg_n_loss = 0
        for fullshot,keypoint,big_five_data in tqdm(train_dataloader):
            middle_frame = fullshot[:, 7, :, :, :]
            middle_frame = middle_frame.permute(0,3,1,2)#batch,chan,h,w
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            # fullshot=imagecrop(fullshot,batchsz)
            middle_frame = middle_frame.to(device)
            big_five_data = big_five_data.to(device)
            
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(middle_frame,keypoint) # 모델의 예측 결과를 저장합니다.
            
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다. 
            optimizer.step()  
            train_avg_loss += loss  
        train_avg_loss=train_avg_loss/len(train_dataloader)
        with torch.no_grad():#validate
            for fullshot,keypoint,big_five_data in tqdm(val_dataloader):
                middle_frame = fullshot[:, 7, :, :, :]
                #4,15,501,3   ------     batch,frame,key,ch
                # fullshot=imagecrop(fullshot,batchsz)
                middle_frame = middle_frame.permute(0,3,1,2)
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                middle_frame = middle_frame.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(middle_frame,keypoint)
                val_loss = criterion(hypothesis, big_five_data)
                
                hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,len(keypoint))
                big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,len(keypoint))
                val_o_loss = criterion(hypothesiso, big_five_datao) 
                val_c_loss = criterion(hypothesisc, big_five_datac) 
                val_e_loss = criterion(hypothesise, big_five_datae) 
                val_a_loss = criterion(hypothesisa, big_five_dataa) 
                val_n_loss = criterion(hypothesisn, big_five_datan)  
                val_avg_loss += val_loss
                val_avg_o_loss += val_o_loss
                val_avg_c_loss += val_c_loss
                val_avg_e_loss += val_e_loss
                val_avg_a_loss += val_a_loss
                val_avg_n_loss += val_n_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
            val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
            val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
            val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
            val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
            val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        start_epoch+=1
        if val_avg_loss < max_value:
            max_value=val_avg_loss
            torch.save({
                'epoch': i+1,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': val_avg_loss,
            }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , 1-Training Loss: {:.4f} , 1-val_avg_o_loss:{:.4f} , 1-val_avg_c_loss:{:.4f} , 1-val_avg_e_loss:{:.4f} , 1-val_avg_a_loss:{:.4f} , 1-val_avg_n_loss:{:.4f}'.format(i+1, 1-val_avg_loss, 1-train_avg_loss,1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))
    

100%|█████████▉| 279/280 [00:51<00:00,  5.78it/s]/home/ssrlab/ENTER/envs/qx/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
100%|██████████| 280/280 [00:52<00:00,  5.36it/s]
[2024-10-15 14:26:11,276]::3947538980::DEBUG::Epoch: 1 , 1-MAE: 0.7157 , 1-Training Loss: 0.7560 , 1-val_avg_o_loss:0.7904 , 1-val_avg_c_loss:0.6196 , 1-val_avg_e_loss:0.7569 , 1-val_avg_a_loss:0.6095 , 1-val_avg_n_loss:0.8023
100%|██████████| 280/280 [00:52<00:00,  5.29it/s]
[2024-10-15 14:32:43,534]::3947538980::DEBUG::Epoch: 2 , 1-MAE: 0.7373 , 1-Training Loss: 0.7680 , 1-val_avg_o_loss:0.8198 , 1-val_avg_c_loss:0.7693 , 1-val_avg_e_loss:0.7971 , 1-val_avg_a_loss:0.5401 , 1-val_avg_n_loss:0.7600
100%|██████████| 280/280 [00:52<00:00, 

In [17]:
#下一个就是使用预训练数据进行实验。
#使用Xception和STGCN各自实验得到结果，进行比较。
#将这个方式用到Video Swin Transformer or Video Vision Transformer